<a href="https://colab.research.google.com/github/cij1012/pytrorch/blob/master/%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_mydata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

당뇨병 진행도 예측

* 독립변수

        -age: 나이
        -sex: 성별
        -bmi: BMI(Body mass index)지수
        -bp: 평균혈압
        -s1~s6: 6종류의 혈액검사수치
* 종속변수

        -1년뒤 측정한 당뇨병의 진행률

In [0]:
import pandas as pd
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

In [2]:
df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
df["target"] = diabetes.target
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [3]:
data = df
X,y = data.values[:,:-1],data.values[:,-1:]
print(X.shape,y.shape)

(442, 10) (442, 1)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [0]:
class MyDataset(Dataset):
  def __init__(self):
    self.x_data = torch.tensor(X, dtype=torch.float)
    self.y_data = torch.tensor(y, dtype=torch.float)
  def __len__(self):
    return len(self.x_data)
  def __getitem__(self,idx):
    x = self.x_data[idx]
    y = self.y_data[idx]
    return x,y

In [0]:
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=True)

In [0]:
model = nn.Linear(10,1)
optimizer = optim.SGD(model.parameters(), lr=0.05)

In [25]:
nb_epochs = 5000
for epoch in range(nb_epochs+1):
  for samples in dataloader:
    x_train, y_train = samples

    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 500 == 0:
      print('Epoch {:4d}/{} Cost : {:.6f}'.format(epoch, nb_epochs,cost.item()))

Epoch    0/5000 Cost : 29141.154297
Epoch  500/5000 Cost : 4578.016602
Epoch 1000/5000 Cost : 3936.876221
Epoch 1500/5000 Cost : 3602.671631
Epoch 2000/5000 Cost : 3408.205566
Epoch 2500/5000 Cost : 3282.618896
Epoch 3000/5000 Cost : 3194.632080
Epoch 3500/5000 Cost : 3129.515625
Epoch 4000/5000 Cost : 3079.680664
Epoch 4500/5000 Cost : 3040.781738
Epoch 5000/5000 Cost : 3010.060059


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [0]:
reg_model = LinearRegression()
reg_model.fit(diabetes['data'],diabetes['target'])
pred_y = reg_model.predict(diabetes['data'])

In [20]:
mean_squared_error(diabetes['target'],pred_y)

2859.6903987680657